In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text as text
from tensorflow.keras import layers
from sklearn.metrics import precision_score,recall_score

In [9]:
data = pd.read_csv("../../Data/clean_data.csv")

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data["Tweet Text"],data["Informativeness"],
test_size=.2,random_state=47,stratify=data["Informativeness"])

In [15]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [16]:
def get_sentence_embedding(sentence):
    preprocessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preprocessed_sentence)
    return bert_embedding["pooled_output"]

### Building a sequential model

In [17]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
text_input = layers.Input(shape=(),dtype=tf.string,name='tweet')

layer = layers.Dense(512, activation="relu", name="hidden")(get_sentence_embedding(text_input)) 
layer = layers.Dense(256, activation="relu", name="hidden")(get_sentence_embedding(text_input)) 
layer = layers.Dense(1, activation="sigmoid", name="output")(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train, y_train,epochs=5)


Epoch 1/5
25/25 [==============================] - 19s 348ms/step - loss: 0.6121 - accuracy: 0.6562
Epoch 2/5
25/25 [==============================] - 9s 344ms/step - loss: 0.4837 - accuracy: 0.7663
Epoch 3/5
25/25 [==============================] - 9s 346ms/step - loss: 0.3872 - accuracy: 0.8438
Epoch 4/5
25/25 [==============================] - 9s 346ms/step - loss: 0.3901 - accuracy: 0.8300
Epoch 5/5
25/25 [==============================] - 9s 346ms/step - loss: 0.3781 - accuracy: 0.8413


In [26]:
model.evaluate(X_test,y_test)

7/7 [==============================] - 2s 344ms/step - loss: 0.3791 - accuracy: 0.8400


In [27]:
prediction = model.predict(X_test)
class_prediction = np.apply_along_axis(convert_to_class,1,prediction)

def convert_to_class(x):
    if(x > .5):
        return 1
    else:
        return 0

In [35]:
# Precision score
precision_score(y_test,class_prediction)

0.8117647058823529

In [36]:
# Recall score
recall_score(y_test,class_prediction)

0.8117647058823529